# Severity of storms 

### 1.1 Business problem


Our project will focus on analyzing and predicting the severity of tornadoes across various regions in America in terms of property damage. To analyze this, we will take the last 10 years of tornado data to represent an approximate measure of it. To calculate this, we will establish a threshold for measuring the severity using the amount of property damage caused by tornadoes. We will predict the property damage of tornadoes and use a scale of low, medium, and high to represent the severity.  
In addition, we will also compare attitudes across each of the 4 seasons - summer, spring, winter, fall, and see if there are any trends present across the 10 year span that shows when tornadoes are most common and damaging. The trends will help us be more accurate in predicting the property damage caused by the tornadoes  per region during different seasons/times of the year.


### 1.2 Datasets


Our dataset details instances of severe weather across a 15 year period. We will be looking at Tornados across this time period. We are given the locations that the Tornados are in, the category of Tornadoes, fatalities, property damage, and the length of the storm. From this we created a target feature called severity which assigns tornados a number from 1 to 10 based on how severe they are.


### 1.3 Proposed analytics solution

 How we get to the target variable -- severity index
The severity index will be calculated by taking into account the property damage, the lives lost in the event, 


In [2]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Read all data into one single dataframe
df_all_data = pd.read_csv('./dataset/storm_event_details_2010.csv')


for i in range(2011,2021):
    df_temp = pd.read_csv(f'./dataset/storm_event_details_{i}.csv')
    df_all_data = df_all_data.append(df_temp, ignore_index=True)



/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_57045/3914789818.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_data = df_all_data.append(df_temp, ignore_index=True)
/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_57045/3914789818.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_data = df_all_data.append(df_temp, ignore_index=True)
/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_57045/3914789818.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_data = df_all_data.append(df_temp, ignore_index=True)
/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_57045/3914789818.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [3]:
## Remove unused columns and format continuous columns

df_hur = df_all_data[df_all_data['EVENT_TYPE']=='Tornado']
df_hur = df_hur.drop(columns=['TOR_OTHER_WFO', 'END_YEARMONTH', 'EVENT_TYPE', 'END_DATE_TIME',
                                           'TOR_OTHER_CZ_STATE','TOR_OTHER_CZ_FIPS','TOR_OTHER_CZ_NAME','DATA_SOURCE','EPISODE_NARRATIVE',
                                            'EVENT_NARRATIVE','WFO','SOURCE','CZ_TIMEZONE','BEGIN_AZIMUTH','END_AZIMUTH','BEGIN_LAT',
                                            'END_LAT','BEGIN_LON','END_LON','STATE_FIPS','BEGIN_RANGE','END_RANGE','DAMAGE_CROPS',
                                            'BEGIN_TIME','END_TIME','BEGIN_LOCATION','END_LOCATION','FLOOD_CAUSE','MAGNITUDE_TYPE',
                                            'MAGNITUDE','CZ_FIPS','CZ_TYPE','CZ_NAME','CATEGORY'])
cols = ['INJURIES_INDIRECT', 'INJURIES_DIRECT', 'DEATHS_INDIRECT', 'DEATHS_DIRECT']
df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)
df_hur['TOR_AREA'] = df_hur['TOR_LENGTH']*df_hur['TOR_WIDTH']
df_hur = df_hur.drop(columns=['TOR_LENGTH', 'TOR_WIDTH'])
df_hur = df_hur.dropna()


/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_57045/2115226861.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)


In [4]:
## Format DAMAGE_PROPERTY column to be float instead of object

dmg = pd.DataFrame(df_hur['DAMAGE_PROPERTY'])

print(dmg)
for index, row in dmg.iterrows():
    val = row['DAMAGE_PROPERTY']
    if val[-1:] == 'B':
        row['DAMAGE_PROPERTY'] = float(val[:-1])*1000000
        # print(float(row['DAMAGE_PROPERTY'][:-1])*1000000)
    elif val[-1:] == 'M':
        row['DAMAGE_PROPERTY'] = float(val[:-1])*1000
        # print(float(row['DAMAGE_PROPERTY'][:-1])*1000)
    elif val[-1:] == 'K':
        row['DAMAGE_PROPERTY'] = float(val[:-1])*1
        # print(float(row['DAMAGE_PROPERTY'][:-1]))

df_hur['DAMAGE_PROPERTY'] = dmg

       DAMAGE_PROPERTY
75               5.00K
304              1.50M
617             10.00K
731            750.00K
732             10.00K
...                ...
688471           0.00K
688472         250.00K
688495          60.00K
688497           0.00K
688560           0.00K

[12945 rows x 1 columns]


In [5]:
## Sort by priority variable and find data split percentages

df_hur = df_hur.sort_values('DAMAGE_PROPERTY', ascending=False)
df_hur.info()
df_hur = df_hur.loc[df_hur['TOR_F_SCALE']!='EFU']
print(df_hur['TOR_F_SCALE'].value_counts()/len(df_hur) * 100)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12945 entries, 100619 to 688560
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BEGIN_YEARMONTH  12945 non-null  int64  
 1   BEGIN_DAY        12945 non-null  int64  
 2   END_DAY          12945 non-null  int64  
 3   EPISODE_ID       12945 non-null  int64  
 4   EVENT_ID         12945 non-null  int64  
 5   STATE            12945 non-null  object 
 6   YEAR             12945 non-null  int64  
 7   MONTH_NAME       12945 non-null  object 
 8   BEGIN_DATE_TIME  12945 non-null  object 
 9   DAMAGE_PROPERTY  12945 non-null  object 
 10  TOR_F_SCALE      12945 non-null  object 
 11  HARM_TOTAL       12945 non-null  int64  
 12  TOR_AREA         12945 non-null  float64
dtypes: float64(1), int64(7), object(5)
memory usage: 1.4+ MB
EF0    51.117296
EF1    35.737575
EF2     9.852883
EF3     2.576541
EF4     0.628231
EF5     0.087475
Name: TOR_F_SCALE, dtype: float

In [6]:
df_hur['MONTH_NAME'].value_counts()/len(df_hur) * 100

April        21.677932
May          20.548708
June         12.413519
July          7.093439
March         6.727634
August        6.067594
October       5.137177
November      4.580517
January       4.413519
February      4.159046
September     3.618290
December      3.562624
Name: MONTH_NAME, dtype: float64

In [7]:
## Split data into strata and sample proportinally (stratified sampling)

sampled_df = df_hur.groupby('TOR_F_SCALE', group_keys=False).apply(lambda x: x.sample(frac=0.1))
sampled_df
# sampled_df['MONTH_NAME'].value_counts()/len(df_hur) * 100

,BEGIN_YEARMONTH,BEGIN_DAY,END_DAY,EPISODE_ID,EVENT_ID,STATE,YEAR,MONTH_NAME,BEGIN_DATE_TIME,DAMAGE_PROPERTY,TOR_F_SCALE,HARM_TOTAL,TOR_AREA
116492,201106,18,18,52812,315753,COLORADO,2011,June,18-JUN-11 15:24:00,0.0,EF0,0,3.00
345728,201507,12,12,96475,595621,MINNESOTA,2015,July,12-JUL-15 16:44:00,0.0,EF0,0,75.00
536903,201808,15,15,130426,780775,INDIANA,2018,August,15-AUG-18 19:07:00,0.0,EF0,0,5.72
559461,201809,16,16,130719,784850,NORTH CAROLINA,2018,September,16-SEP-18 00:51:00,5.0,EF0,0,5.10
676440,202012,24,24,154311,929772,MISSISSIPPI,2020,December,24-DEC-20 01:07:00,15.0,EF0,0,168.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123535,201104,27,27,50516,301925,ALABAMA,2011,April,27-APR-11 15:01:00,0.0,EF4,6,30466.80
378425,201512,23,23,102065,610234,MISSISSIPPI,2015,December,23-DEC-15 16:10:00,5.0,EF4,0,3120.00
377874,201512,23,23,102042,610239,TENNESSEE,2015,December,23-DEC-15 17:03:00,227.0,EF4,0,9217.00
312392,201404,27,27,83594,504760,ARKANSAS,2014,April,27-APR-14 18:25:00,210000.0,EF4,179,31996.80


In [37]:
cont_features = []
for (columnName, columnData) in sampled_df.iteritems():
    if type(columnData) == int or float:
        cont_features.append(columnName)
        
print(cont_features)

['BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_DAY', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'YEAR', 'DAMAGE_PROPERTY', 'HARM_TOTAL', 'TOR_AREA']


In [43]:
df2 = sampled_df[cont_features]
df2.drop('STATE', axis = 1, inplace = True)
df2

,BEGIN_YEARMONTH,BEGIN_DAY,END_DAY,EPISODE_ID,EVENT_ID,YEAR,DAMAGE_PROPERTY,HARM_TOTAL,TOR_AREA
116492,201106,18,18,52812,315753,2011,0.0,0,3.00
345728,201507,12,12,96475,595621,2015,0.0,0,75.00
536903,201808,15,15,130426,780775,2018,0.0,0,5.72
559461,201809,16,16,130719,784850,2018,5.0,0,5.10
676440,202012,24,24,154311,929772,2020,15.0,0,168.50
...,...,...,...,...,...,...,...,...,...
123535,201104,27,27,50516,301925,2011,0.0,6,30466.80
378425,201512,23,23,102065,610234,2015,5.0,0,3120.00
377874,201512,23,23,102042,610239,2015,227.0,0,9217.00
312392,201404,27,27,83594,504760,2014,210000.0,179,31996.80


In [44]:
from sklearn import preprocessing
scaled_df = preprocessing.MinMaxScaler().fit_transform(df2)
sdf=pd.DataFrame(scaled_df, columns=df2.columns)
#sdf['price_range']=cl_df['prie_range']
sdf.head()

,BEGIN_YEARMONTH,BEGIN_DAY,END_DAY,EPISODE_ID,EVENT_ID,YEAR,DAMAGE_PROPERTY,HARM_TOTAL,TOR_AREA
0,0.102081,0.566667,0.566667,0.139431,0.145627,0.1,0.00000,0.0,0.000056
1,0.499504,0.366667,0.366667,0.507219,0.532097,0.5,0.00000,0.0,0.001457
2,0.797820,0.466667,0.466667,0.793199,0.787776,0.8,0.00000,0.0,0.000109
3,0.798811,0.500000,0.500000,0.795667,0.793403,0.8,0.00001,0.0,0.000097
4,1.000000,0.766667,0.766667,0.994390,0.993526,1.0,0.00003,0.0,0.003277


In [2]:
# visualize damage property scale of tornado with a scattarplot and 
#tor_f_scale 

#df_hur.dropna()
#df_hur = df_hur.astype({'DAMAGE_PROPERTY':'float'})

#df_hur['logarithm_base10'] = np.log10(df_hur['DAMAGE_PROPERTY'])
#df_hur['natural_log'] = np.log(df_hur['DAMAGE_PROPERTY'])
#plt.scatter(df_hur.DAMAGE_PROPERTY, df_hur.HARM_TOTAL, linewidths = 2, edgecolor ="green", s = 1, hue = 'TOR_F_SCALE')


sns.set(rc={'figure.figsize':(12, 8)})
sns.scatterplot(data=sdf, x='DAMAGE_PROPERTY', y='HARM_TOTAL', palette='deep')
#robust scaling 


plt.xlabel('property damage', fontsize=10)
plt.ylabel('total harm', fontsize=10)
plt.xlim(0.1)
plt.show()


correlation = df_hur.corr()
print(correlation.loc['DAMAGE_PROPERTY', 'HARM_TOTAL'])


#visualize tornado area 

NameError: name 'sns' is not defined

### Example Data Quality Report for Continuous Variables
| Feature | Desc. | Count | % of Missing | Card. | Min. | Q1 | Median | Q3 | Max. | Mean | Std. Dev. | Notes |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| AAGE | Age | 199,523 | 0 | 91 | 0 | 15 | 33 | 50 | 90 | 34.49 | 22.31 |  |

### Example Data Quality Report for Categorical Variables
| Feature | Desc. | Count | % of Missing | Card. | Mode | Mode Freq. | Mode % | 2nd Mode | 2nd Mode Freq. | 2nd Mode Perc | Notes |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| ACLSWKR | Class of worker | 199,523 | 0 | 9 | Not in Universe | 100,245 | 50.24 | Private | 72,028 |  36.10 |  |

### Missing values and outliers

For missing values we will be dropping the instances because we will be taking a 10% sample of the instances. for outliers if its skewed then the outliers are more important. 

### Normalization

what we need to normalize and how we will normalize it

### Transformation

a way to deal with outliers 